# Aftermarket Toolkit - Notification System Documentation

Comprehensive documentation of the notification system implementation for the Aftermarket Toolkit application.

## Table of Contents

1. [Overview](#Overview)
2. [Features](#Features)
3. [System Architecture](#System-Architecture)
4. [Implementation Details](#Implementation-Details)
   - [Database Schema](#Database-Schema)
   - [Core Functions](#Core-Functions)
   - [Frontend Integration](#Frontend-Integration)
5. [Notification Types](#Notification-Types)
6. [Email Notifications](#Email-Notifications)
7. [Migration to MVC](#Migration-to-MVC)
8. [Testing](#Testing)
9. [Future Enhancements](#Future-Enhancements)

## Overview

The notification system is a core component of the Aftermarket Toolkit application that provides real-time updates to users about important events such as messages, forum replies, friend requests, and listing comments. The system includes both in-app notifications and email notifications.

The notification system is designed to be:

- **Real-time**: Notifications appear immediately when events occur
- **Persistent**: Notifications are stored in the database
- **Cross-page**: Notifications are accessible from anywhere in the application
- **Multi-channel**: Notifications can be delivered in-app and via email

## Features

- **Multiple notification types**: Support for different types of notifications (forum responses, messages, friend requests, listing comments)
- **Real-time notification display**: Notifications appear in real-time in the notification dropdown
- **Notification badge**: Shows the number of unread notifications
- **Mark as read**: Users can mark individual or all notifications as read
- **Email notifications**: Optional email notifications for important events
- **User preferences**: Users can enable/disable email notifications
- **Notification history**: Users can view all past notifications
- **Time-based formatting**: Notifications show how long ago they were created

## System Architecture

The notification system consists of the following components:

1. **Database Layer**: Stores notifications in the `notifications` table
2. **Backend PHP Functions**: Handles creating, retrieving, and managing notifications
3. **Email Notification System**: Sends email notifications when configured
4. **Frontend JavaScript**: Displays notifications in the UI and handles user interactions
5. **CSS Styling**: Provides visual styling for the notification components

### Component Interaction Flow

1. **Event Trigger**: User action or system event (e.g., forum reply, message)
2. **Notification Creation**: Backend creates a notification record
3. **Email Notification**: Optional email is sent if user has enabled it
4. **Notification Fetching**: Frontend regularly polls for new notifications
5. **UI Update**: Notification badge and dropdown are updated
6. **User Interaction**: User can view, mark as read, or delete notifications

## Implementation Details

### Database Schema

```sql
CREATE TABLE notifications (
  id INT AUTO_INCREMENT PRIMARY KEY,
  user_id INT NOT NULL,
  type VARCHAR(50) NOT NULL,
  sender_id INT NULL,
  related_id INT NULL,
  content TEXT NOT NULL,
  link VARCHAR(255) NULL,
  is_read TINYINT(1) DEFAULT 0,
  created_at DATETIME NOT NULL,
  FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
);
```

### Core Functions

#### Creating Notifications

Notifications are created using the `sendNotification()` function in `notification_handler.php`:

```php
function sendNotification($conn, $userId, $type, $senderId = null, $relatedId = null, $content = '', $link = null) {
    // Validate inputs and prepare notification
    // Insert notification into database
    // Send email notification if enabled
}
```

#### Retrieving Notifications

Notifications are retrieved using the `getUserNotifications()` function:

```php
function getUserNotifications($conn, $userId, $limit = 10, $unreadOnly = false) {
    // Query database for notifications
    // Enhance notifications with additional details
    // Return formatted notifications
}
```

#### Marking Notifications as Read

Notifications can be marked as read using:

```php
function markNotificationAsRead($conn, $notificationId, $userId) {
    // Update notification status in database
}

function markAllNotificationsAsRead($conn, $userId, $type = null) {
    // Update all notifications for user
}
```

### Frontend Integration

The notification system integrates with the frontend using JavaScript in `notifications.js`:

- Regular polling for new notifications using AJAX
- Updating the notification badge count
- Displaying notifications in the dropdown
- Handling user interactions (marking as read, navigating to notification content)

## Notification Types

The system supports the following notification types:

### 1. Forum Responses

Triggered when someone replies to a user's forum thread.

**Implementation**:
- Created in `forum_response_handler.php`
- Icon: Comments icon
- Links to the relevant forum thread

### 2. Chat Messages

Triggered when a user receives a new chat message.

**Implementation**:
- Created in `chat_message_handler.php` and `enhanced_chat_message_handler.php`
- Icon: Envelope icon
- Links to the chat with the sender
- Message content is not included in email notifications for privacy

### 3. Friend Requests

Triggered when someone sends a friend request.

**Implementation**:
- Created in friend request handling code
- Icon: User-plus icon
- Links to the friends page

### 4. Listing Comments

Triggered when someone comments on a user's marketplace listing.

**Implementation**:
- Created in listing comment code
- Icon: Comment icon
- Links to the relevant listing

## Email Notifications

The system supports sending email notifications for important events if the user has enabled them.

### Implementation

Email notifications are handled by the `sendNotificationEmail()` function in `notification_email.php`:

```php
function sendNotificationEmail($userId, $notificationType, $content, $conn) {
    // Check if user has email notifications enabled
    // Create appropriate email subject and content
    // Send email using PHPMailer
}
```

### Email Templates

Emails are sent using HTML templates with the following features:
- Responsive design that works on desktop and mobile
- Styled according to application branding
- Icons for different notification types
- Links back to the relevant content
- Privacy protection (message content not included in emails)

### User Preferences

Users can enable or disable email notifications in their profile settings. This preference is stored in the `email_notifications` column in the `users` table.

## Migration to MVC

As part of the planned migration to the MVC (Model-View-Controller) architecture, the notification system will be restructured as follows:

### Model

```php
// app/models/Notification.php
class Notification {
    private $db;
    
    public function __construct($db) {
        $this->db = $db;
    }
    
    public function create($userId, $type, $senderId, $relatedId, $content, $link = null) {
        // Create notification logic
    }
    
    public function getForUser($userId, $limit = 10, $unreadOnly = false) {
        // Get notifications logic
    }
    
    public function markAsRead($id, $userId) {
        // Mark as read logic
    }
    
    public function markAllAsRead($userId, $type = null) {
        // Mark all as read logic
    }
}
```

### Controller

```php
// app/controllers/NotificationController.php
class NotificationController {
    private $notificationModel;
    private $emailService;
    
    public function __construct($db) {
        $this->notificationModel = new Notification($db);
        $this->emailService = new EmailService($db);
    }
    
    public function index() {
        // Show all notifications
    }
    
    public function getLatest() {
        // Get latest notifications (AJAX endpoint)
    }
    
    public function markAsRead($id) {
        // Mark notification as read
    }
    
    public function markAllAsRead() {
        // Mark all notifications as read
    }
}
```

### View

```php
<!-- app/views/notifications/index.php -->
<div class="notifications-container">
    <h1>Your Notifications</h1>
    
    <?php foreach ($notifications as $notification): ?>
        <div class="notification-item <?= $notification['is_read'] ? '' : 'unread' ?>">
            <!-- Notification display -->
        </div>
    <?php endforeach; ?>
</div>
```

## Testing

The notification system can be tested using the following methods:

### 1. Manual Testing

Test the notification system by performing actions that trigger notifications:

- Send a chat message to another user
- Reply to a forum thread
- Send a friend request
- Comment on a marketplace listing

### 2. Test Scripts

Use the provided test scripts:

- `notification-email-test.php`: Tests email notification delivery
- `chat_fix_test.php`: Tests chat notification system
- `update_chat_notifications.php`: Updates and tests the enhanced chat notifications

### 3. Email Testing

Test email notifications by:

1. Ensuring email notifications are enabled in user profile
2. Performing actions that trigger notifications
3. Checking the recipient's email inbox
4. Verifying the email format and content

## Future Enhancements

### 1. Real-time Notifications

Implement WebSockets for real-time notifications instead of polling:

```javascript
// Initialize WebSocket connection
const socket = new WebSocket('ws://your-websocket-server');

// Listen for notifications
socket.addEventListener('message', function(event) {
    const notification = JSON.parse(event.data);
    updateNotificationUI(notification);
});
```

### 2. Notification Grouping

Group similar notifications to reduce clutter:

```php
function groupNotifications($notifications) {
    $grouped = [];
    $counts = [];
    
    foreach ($notifications as $notification) {
        // Group logic
    }
    
    return $grouped;
}
```

### 3. Advanced Filtering

Allow users to filter notifications by type, date, or other criteria:

```php
function getFilteredNotifications($userId, $filters) {
    $sql = "SELECT * FROM notifications WHERE user_id = ?";
    
    // Add filter conditions
    if (!empty($filters['type'])) {
        $sql .= " AND type = ?";
    }
    
    if (!empty($filters['from_date'])) {
        $sql .= " AND created_at >= ?";
    }
    
    // Execute query
}
```

### 4. Notification Analytics

Implement analytics to track notification engagement:

```php
function trackNotificationEngagement($notificationId, $action) {
    // Record when user interacts with notification
}
```

## Conclusion

The notification system is a critical component of the Aftermarket Toolkit platform, keeping users informed about relevant activities. The current implementation provides a solid foundation with support for different notification types, in-app notifications, and email notifications.

The planned migration to MVC architecture will further improve the system's maintainability and extensibility, making it easier to add new notification types and channels in the future.

Regular testing and monitoring of the notification system is recommended to ensure it continues to function properly as the application evolves.